In [102]:
import numpy as np

In [103]:
states = [1,2,3,4,5,6,7,8,9]
actions = ['droite', 'haut','gauche','bas']

In [104]:
def transition_matrix(u):
    if u == 'gauche':
        mini = np.array([[1,0,0],[1,0,0],[0,1,0]])
        zeros_block = np.array([[0,0,0],[0,0,0],[0,0,0]])
        tr = np.block([[mini, zeros_block, zeros_block], [zeros_block,mini, zeros_block],[zeros_block,zeros_block,mini]])
    
    if u == 'droite':
        mini = np.array([[0,1,0],[0,0,1],[0,0,1]])
        zeros_block = np.array([[0,0,0],[0,0,0],[0,0,0]])
        tr = np.block([[mini, zeros_block, zeros_block], [zeros_block,mini, zeros_block],[zeros_block,zeros_block,mini]])
        
    if u == 'haut':
        mini = np.eye(3)
        zeros_block = np.array([[0,0,0],[0,0,0],[0,0,0]])
        tr = np.block([[mini, zeros_block, zeros_block], [mini, zeros_block, zeros_block],[zeros_block,mini,zeros_block]])
        
    if u == 'bas':
        mini = np.eye(3)
        zeros_block = np.array([[0,0,0],[0,0,0],[0,0,0]])
        tr = np.block([[zeros_block, mini, zeros_block], [zeros_block, zeros_block, mini],[zeros_block,zeros_block, mini]])    
    return tr

In [105]:
def transition(s, u):
    tr = transition_matrix(u)
    ligne = tr[s - 1]
    return np.random.choice(states, size=1, replace=True, p=ligne)[0]

In [158]:
def reward(s, u):
#     if ((s == 2) and (u=='droite')) or (s==3) or ((s==6) and (u=='haut')):
#         return(10)
#     elif ((s==4)) or ((s==1) and (u=='bas')) or ((s==5) and (u=='gauche')) or ((s==7) and (u == 'haut')):
#         return(-10)
    if s==3:
        return 10
    elif s==4:
        return -10
    else:
        return 0

In [153]:
def  env_step(s, u):
    next_state = transition(s,u)
    rew = reward(s, u)
    done = (s == 3)
    return next_state, rew, done

# Q-learning algorithm

In [154]:
q_table = np.zeros([len(states),len(actions)])

In [159]:
# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

In [162]:
for i in range(1, 10000):
    s = np.random.randint(1,len(states)+1) #initial state
    state_index = s-1
    
    epochs, penalties, rew = 0, 0, 0
    done = False
    
    while not done:
        if (s==3):
            done = True
            rew = env_step(s, 'bas')#n'importe quelle action
            
        else:
            if np.random.uniform(0, 1) < epsilon:
                action_index = np.random.randint(0,len(actions)) # Explore action space
            else:
                action_index = np.argmax(q_table[state_index]) # Exploit learned values

            u = actions[action_index]

            next_state, rew, done = env_step(s, u)
            next_state_index = next_state - 1

            old_value = q_table[state_index, action_index]
            next_max = np.max(q_table[next_state_index])

            new_value = (1 - alpha) * old_value + alpha * (rew + gamma * next_max)
            q_table[state_index, action_index] = new_value

            if rew == -10:
                penalties += 1

            s = next_state
            state_index = s-1

        epochs += 1


print("Training finished.\n")

Training finished.



In [163]:
print(actions)
print(q_table)

['droite', 'haut', 'gauche', 'bas']
[[  9.           5.2612627    5.28236425  -2.7456397 ]
 [ 15.           8.99226026   5.39401553   5.39853449]
 [ 25.          23.19652531  17.57958464  16.72137917]
 [ -4.6         -4.61380276 -12.57732192  -7.99489779]
 [  9.           8.99573381  -2.75390342   3.23864383]
 [  9.          15.           5.4          5.4       ]
 [  3.24        -2.68323656   1.92141055   1.91834253]
 [  5.4          5.3971021    1.94348803   3.23790099]
 [  5.39993647   9.           3.23996696   5.39994978]]
